In [200]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from stilus.data.sets import MidiDataset
from torch.utils.data import DataLoader

In [201]:
class ConvNet(nn.Module):

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(5, 10, 4 )
        self.conv2 = nn.Conv1d(10, 10, 4)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(10 * 13, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool1d(x,2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


In [202]:
conv_net = ConvNet()
print(conv_net)

ConvNet(
  (conv1): Conv1d(5, 10, kernel_size=(4,), stride=(1,))
  (conv2): Conv1d(10, 10, kernel_size=(4,), stride=(1,))
  (fc1): Linear(in_features=130, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=5, bias=True)
)


In [203]:
input = torch.randn(128, 5, 32)
out = conv_net(input)
print(input.shape)
print(out.shape)

torch.Size([128, 5, 32])
torch.Size([128, 5])


In [211]:
criterion = nn.L1Loss()
optimizer = optim.SGD(conv_net.parameters(), lr=0.001, momentum=0.9)

In [205]:
midi_dataset = MidiDataset("training_data.npy")
dataloader = DataLoader(midi_dataset, batch_size=32, shuffle=True)

In [212]:
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[:,:,0:32], data[:,:,32]

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = conv_net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 1000 == 999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Finished Training')

[1,  1000] loss: 11.461
[1,  2000] loss: 10.898
[1,  3000] loss: 10.464
[1,  4000] loss: 10.244
[1,  5000] loss: 10.057
[1,  6000] loss: 9.762
[1,  7000] loss: 9.642
[1,  8000] loss: 9.465
[2,  1000] loss: 9.434
[2,  2000] loss: 9.430
[2,  3000] loss: 9.394
[2,  4000] loss: 9.408
[2,  5000] loss: 9.304
[2,  6000] loss: 9.397
[2,  7000] loss: 9.304
[2,  8000] loss: 9.380
[3,  1000] loss: 9.264
[3,  2000] loss: 9.281
[3,  3000] loss: 9.322
[3,  4000] loss: 9.248
[3,  5000] loss: 9.190
[3,  6000] loss: 9.071
[3,  7000] loss: 9.160
[3,  8000] loss: 9.188
[4,  1000] loss: 9.021
[4,  2000] loss: 9.158
[4,  3000] loss: 9.064
[4,  4000] loss: 9.033
[4,  5000] loss: 9.068
[4,  6000] loss: 9.125
[4,  7000] loss: 8.995
[4,  8000] loss: 8.937
[5,  1000] loss: 8.976
[5,  2000] loss: 8.927
[5,  3000] loss: 8.928
[5,  4000] loss: 8.927
[5,  5000] loss: 8.857
[5,  6000] loss: 8.820
[5,  7000] loss: 8.689
[5,  8000] loss: 8.924
Finished Training


In [216]:
for i, data in enumerate(dataloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data[0:2,:,0:32], data[0:2,:,32]
    print("pred:", conv_net(inputs))
    print("labels:", labels)
    if i == 2:
        break

pred: tensor([[-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00],
        [-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00]],
       grad_fn=<AddmmBackward>)
labels: tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
pred: tensor([[-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00],
        [-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00]],
       grad_fn=<AddmmBackward>)
labels: tensor([[ 0.,  0.,  0., 63., 75.],
        [ 0.,  0.,  0.,  0.,  0.]])
pred: tensor([[-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00],
        [-1.0344e-04,  3.5378e-04, -3.3060e-04,  4.3155e-04,  7.8967e+00]],
       grad_fn=<AddmmBackward>)
labels: tensor([[ 0.,  0.,  0.,  0.,  0.],
        [ 0.,  0., 69., 72., 75.]])
